# Network Details & Terminology

![neural network notation and terminology](nn.png)

## Points to Remember
1. **Weights and Biases**

- Between any 2 layers $l$ and $(l-1)$, there are $n_l * n_{(l-1)}$ connections. For example, layer $1$ has $n_1 = 3$ nodes and layer $0$ has $n_0 = 2$ nodes. Thus, there are a total of $3 * 2$ connections (each node in $l=1$ has $n_0 = 2$ connections) & corresponding weights. This makes the dimension of the weight matrix, $n_l * n_{(l-1)}$.

- Bias vectors are only applied to the hidden and output layers. They have a dimension of $n_l * 1$.

3. **Sample Size**

- The input $X$ and output matrices, $Z$ and $A$ have dimensions $n_l * m$ where $m$ is the total number of training samples. If we were dealing with a single training instance, i.e., $m=1$, each matrix would just become a column vector. _We structure our matrices like this to make the math easier._

In [ ]:
# Literally the ONLY library we need to implement our network!
import numpy as np

### Initializing our Weights/Biases acc. to the dimension logic

In [ ]:
# Implementing the architecture
L = 3
n = [2, 3, 3, 1]
print(f"Number of Layers: {L}")
for idx in range(len(n)):
    print(f"Nodes in layer {idx} : {n[idx]}")

# Dimensions of my weight matrices
dim_W1 = (n[1], n[0]) # Weights connecting layer 1 & 0
dim_W2 = (n[2], n[1]) # Weights connecting layer 2 & 1
dim_W3 = (n[3], n[2]) # Weights connecting layer 3 & 2

# Randomly initializing my weight matrices
W1 = np.random.randn(*dim_W1)
W2 = np.random.randn(*dim_W2)
W3 = np.random.randn(*dim_W3)

# Dimensions of my bias vectors
dim_B1 = (n[1], 1)
dim_B2 = (n[2], 1)
dim_B3 = (n[3], 1)

# Randomly initializing my bias vectors
B1 = np.random.rand(*dim_B1)
B2 = np.random.rand(*dim_B2)
B3 = np.random.rand(*dim_B3)

### Moving through the network

Any neural network has 2 phases of data manipulation,

1. Forward Pass: Sequentially transforming the input data through the network to get the final prediction vector.
2. Backward Pass: Computing the gradient of the cost function w.r.t the model parameters & updating the parameters using the calculated gradient.

### Forward Pass

![forward pass for one layer example](forward_pass.png)

I'll explain forward-pass in 3 steps. 
1. Show calculations for each hidden node.
2. Vectorize them, i.e., packaging everything nicely in vectors/matrices.
3. Generalize to $m$ samples.

**A. Show calculations for each hidden node.**

Each non-input layer node does 2 things,
* Multiplies the values of the previous nodes with the corresponding weights & sums it up with the biases. This gives the $Z$ vector output.
* Applies a non-linear _activation_ function to the previous sum. This gives the $A$ vector output.

#### Side note on Activation Functions

Without an activation function, the entire network would be a collection of linear transformations without learning anything useful. When choosing a function for our network, we want it to have desirable properties such as being differentiable, having global minimums, etc. Popular choices include ReLU, Sigmoid, TanH, etc. We chose sigmoid for this tutorial.

*sigmoid Function*
Let $X$ be the input (a vector or a single number). The sigmoid function is then given by,
* $sigmoid(X) = 1/(1 + e^{-X})$

In [1]:
def sigmoid(X):
    return 1/(1 + np.exp(-X))

### Back to Forward Pass

Let $z_1$, $z_2$, and $z_3$ be the _pre-activation_ outputs for each node in the hidden layer above. Additionally, we apply the biases, $0.8, 0.1, 0.3$ respectively. This gives us,

* $z_1 = (3 * 2 + 6 * 5) + 0.8 = 36.8$
* $z_2 = (4 * 2 + 7 * 5) + 0.1 = 43.1$
* $z_3 = (5 * 2 + 8 * 5) + 0.3 = 50.3$

Looking at the sum, we see that we're basically just performing this expression, $weight * input + bias$.

Let $a_1$, $a_2$, and $a_3$ be the _activated_ outputs for each node in the hidden layer above. This gives us,

* $a_1 = sigmoid(36.8) = 1.0$
* $a_2 = sigmoid(43.1) = 1.0$
* $a_3 = sigmoid(50.3) = 1.0$

**B. Vectorize them**

While it is possible to do all of these calculations independently, it does not scale well to a large number of weights. Additionally, we're not taking advantage of our GPU's which are specifically designed to be good at handling matrix/vector math. So, let's package everything to address this.

Let $X$ be our input vector, $W$ our weight matrix, $B$ the bias vector, $Z$ be our pre-activated output & $A$ our activated output. Thus,

* $X = \begin{bmatrix}
 2 \\
 5
\end{bmatrix}$

* $W = \begin{bmatrix}
 3 & 6 \\
 4 & 7 \\
 5 & 8 \\
\end{bmatrix}$

* $B = \begin{bmatrix}
 0.8 \\
 0.1 \\
 0.3
\end{bmatrix}$

$\therefore W * X = 
\begin{bmatrix}
 3 & 6 \\
 4 & 7 \\
 5 & 8 \\
\end{bmatrix} * 
\begin{bmatrix}
 2 \\
 5
\end{bmatrix} = 
\begin{bmatrix}
(3 * 2 + 6 * 5)\\
(4 * 2 + 7 * 5)\\ 
(5 * 2 + 8 * 5)
\end{bmatrix}$

$W * X + B = \begin{bmatrix}
(3 * 2 + 6 * 5)\\
(4 * 2 + 7 * 5)\\ 
(5 * 2 + 8 * 5)
\end{bmatrix} + 
\begin{bmatrix}
 0.8 \\
 0.1 \\
 0.3
\end{bmatrix} = \begin{bmatrix}
(3 * 2 + 6 * 5) + 0.8\\
(4 * 2 + 7 * 5) + 0.1\\ 
(5 * 2 + 8 * 5) + 0.3
\end{bmatrix} = 
\begin{bmatrix}
36.8 \\ 43.1 \\ 50.3
\end{bmatrix} = Z$

* Applying sigmoid to $Z$, we get,

$A = \begin{bmatrix}
sigmoid(36.8) \\
sigmoid(43.1)\\
sigmoid(50.3)
\end{bmatrix} = 
\begin{bmatrix}
1.0 \\
1.0 \\
1.0
\end{bmatrix}$

<span style="color:green">_You can see these equations are the same as what we got above!_</span>.

In general, we have the following 2 equations between any pair of consecutive nodes,

$Z_l = W_l * A_{l-1} + B_l$ and $A_l = activation(Z_l)$

**C. Generalize to $m$ samples.**

Whatever we did above, was for a single input or *1 sample*. We usually deal with much more samples when training our network. The nice thing is, we don't really have to edit much to extend this idea to $m$ samples since we're taking advantage of **matrix multiplication**.

Say, our input is now, $X = \begin{bmatrix}
2 & -1 & 7 \\
5 & -3 & 7 \\
\end{bmatrix}$, i.e., we now have 2 more samples alongside our original $\begin{bmatrix}2 \\ 5\end{bmatrix}$.

$X$ has a dimension of $2 * 3$ now. See how easily it aligns with the above equations? The weight matrix has a dimension of $3 * 2$. Thus, the resultant matrix will have a dimension of: $(3 * 2) * (2 * 3) = (3 * 3)$. The bias vector is just repeated 3 times (broadcasted) to become a $(3 * 3)$ matrix as well. This shows how easily we can extend these calculations to $m = 3, 10, \ldots$ samples.

In [ ]:
#Feed-forward layer-by-layer
def feed_forward(X):
    # Layer 1 calculations
    Z1 = W1 @ X + B1
    A1 = sigmoid(Z1)
    
    # Layer 2 calculations
    Z2 = W2 @ A1 + B2
    A2 = sigmoid(Z2)
    
    # Layer 3 calculations
    Z3 = W3 @ A2 + B3
    A3 = sigmoid(Z3) # y_hat (final prediction)

    # Returning the activations & weights since we need them for during our derivative calculations.
    activations = {"A1": A1, "A2": A2}
    weights = {"W1": W1, "W2": W2, "W3": W3}
    
    return A3, activations, weights

### So how did we do? - Loss functions

To gauge how well our network is doing, we compute a _loss_ function which tells us how far the network predictions are from the true values. There are several functions we can use, depending on the task. However, since we are dealing with a 2-class problem, **binary cross-entropy** is a good choice for such problems.

_Cost v/s Loss_: While they are used interchangeably, _cost_ measures how well the network is doing w.r.t **ALL** training samples while _loss_ is measured on a per sample basis.

Let $\hat{y}$ be the network prediction for a given sample whose true class is $y$. The BCE loss is then given by,

![Binary Cross Entropy](bce.png)

If $Y$ is the true output vector & $\hat{Y}$ is the prediction probability vector, 

The BCE **cost** is then given by, 

$C_{BCE} (\hat{Y}, Y) =  -1/m * [Y * \log\hat{Y} + (1 - Y)*\log(1-\hat{Y})]$

i.e., we're simply just taking an average across all the predictions. We apply a negative sign in front of the expression to turn negative log values positive.

Let's work through an example & then code it up! Say we have the following, 

$Y = \begin{bmatrix}1 \\ 1 \\ 0 \end{bmatrix}$ $\hat{Y} = \begin{bmatrix}0.7 \\ 0.2 \\ 0.9 \end{bmatrix}$

First part of the equation gives us, 

$Y * \log(\hat{Y}) = \begin{bmatrix}1 \\ 1 \\ 0 \end{bmatrix} * \log{\begin{bmatrix}0.7 \\ 0.2 \\ 0.9 \end{bmatrix}} = \begin{bmatrix}1 \\ 1 \\ 0 \end{bmatrix} * \begin{bmatrix}\log{0.7} \\ \log{0.2} \\ \log{0.9} \end{bmatrix} = \begin{bmatrix}\log{0.7} \\ \log{0.2} \\ 0 \end{bmatrix}$

**Notice here that we just did element-wise multiplication rather than matrix multiplication as we ultimately want a single number representing the loss.**

Second part of the equation gives us (we **broadcast** 1 to align with the vectors),

$(1 - Y) * \log(1 - \hat{Y}) = \left( \begin{bmatrix}1 \\ 1 \\ 1 \end{bmatrix} - \begin{bmatrix}1 \\ 1 \\ 0 \end{bmatrix} \right) * \log{\left( \begin{bmatrix}1 \\ 1 \\ 1 \end{bmatrix} - \begin{bmatrix}0.7 \\ 0.2 \\ 0.9 \end{bmatrix} \right)} = \begin{bmatrix}0 \\ 0 \\ 1 \end{bmatrix} * \log{\left( \begin{bmatrix}(1-0.7) \\ (1-0.2) \\ (1-0.9) \end{bmatrix} \right)} = \begin{bmatrix}0 \\ 0 \\ 1 \end{bmatrix} * \left( \begin{bmatrix}\log{(1-0.7)} \\ \log{(1-0.2)} \\ \log{(1-0.9)} \end{bmatrix} \right) = \begin{bmatrix}0 \\ 0 \\ \log{(1-0.9)} \end{bmatrix}$

Adding the two, we get, 

$\begin{bmatrix}\log{0.7} \\ \log{0.2} \\ 0 \end{bmatrix} + \begin{bmatrix}0 \\ 0 \\ \log{(1-0.9)} \end{bmatrix} = \begin{bmatrix}\log{0.7} \\ \log{0.2} \\ \log{(1-0.9)} \end{bmatrix} = -[\log{0.7} + \log{0.2} + \log{(1-0.9)}]/3$ (We're taking an average of the elements and prepending the negative sign). 

This is the exact same value we would get if we did cross-entropy loss for each sample. Example,

$-([1 * \log{0.7} + (1 - 1) * \log{(1 - 0.7)}] + $

$[1 * \log{0.2} + (1 - 1) * \log{(1 - 0.2)}] + $

$[0 * \log{0.9} + (1 - 0) * \log{(1 - 0.9)}])/3 = -[\log{0.7} + \log{0.2} + \log{(1 - 0.9)}]/3$

This again illustrates the interplay between vectors and individual equations.

In [ ]:
def bce_loss(y_true: np.ndarray, y_pred:np.ndarray, epsilon: float = 1e-12) -> float:
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    '''
    One way to implement it is using matrix multiplication - This will give us the final sum. But we need to explicitly divide by num. of samples to get the mean.
    loss = -(y_true @ np.log(y_pred) + (1 - y_true) @ np.log(1 - y_pred))/y_true.shape[0]
    '''
    
    # Another equivalent way (using element-wise multiplication: This will give us individual vectors that need to be summed)
    loss = -np.mean(y_true*np.log(y_pred) + (1 - y_true)*np.log(1 - y_pred))

    '''
    Sometimes loss1 might not be equal to loss2. Although they are both equivalent formulations, matrix multiplication sometimes
    introduces some rounding errors. But it is negligible to be bothered about.
    '''
    
    return loss.item()

### Backpropagation

Finally, we concluded the forward-pass &#x1F601; Now for the actual challenging part, backprop &#x1F61F; This is the phase where we tune our model parameters (weights & biases) to get optimal values. We do this by taking the gradient of the cost function, w.r.t. each model parameter. The gradient w.r.t a given parameter tells us if we need to increase or decrease its value to reduce the cost. For example, if $\frac{\partial C}{\partial w} = 2$, it means changing $w$ by $\partial w$ will increase $C$ by $2 * \partial w$. As such, we will need to reduce the value of $w$ to decrease $C$.

The cost $C$ is a function of $\hat{Y}$ and $Y$. While the latter is a constant (just the labels), $\hat{Y}$ is the prediction vector which is essentially a function of all network parameters ($w_{11}, b_{11} \ldots$). However, taking derivatives w.r.t each, 1-by-1 would be computationally infeasible. As such, we employ matrix tricks like we've done before. This means, in our case, there are _3 weights ($W_1, W_2, W_3$) & 3 biases ($B_1, B_2, B_3$)_ we need to deal with.

As we have 3 weights & 3 biases or **6 parameters**, we need **6 partial derivatives**: $\Bigl[ \frac{\partial C}{\partial W_1}, \frac{\partial C}{\partial W_2}, \frac{\partial C}{\partial W_3}, \frac{\partial C}{\partial B_1}, \frac{\partial C}{\partial B_2}, \frac{\partial C}{\partial B_3} \Bigr]$.

<span style="color:blue">Why partial derivatives?</span>
Ans: Partial derivatives allow us to isolate the terms of interest (in our case the 6 above) by treating everything else as constants.

The following is the full chain of partial derivatives that we need to consider. We'll work through each partial derivative & code them up immediately. You will see that it makes more sense to work backwards from the last to first layer to get these values (another reason why its called _backprop_).

![full chain of derivatives](chain.png)

#### Partial Derivatives coming from layer 3

![layer 3 derivatives](layer3.png)

To get $\frac{\partial C}{\partial W_3}$ and $\frac{\partial C}{\partial B_3}$, we apply the multiplication (chain) rule along the <span style="color:orange">orange</span> path to get the partial derivatives.

For instance, $\frac{\partial C}{\partial W_3} =  \frac{\partial C}{\partial A_3} * \frac{\partial A_3}{\partial Z_3} * \frac{\partial Z_3}{\partial W_3}$

Let's solve each part of the above equation one-by-one.

$C = -1/m [Y*\log{A_3} + (1 - Y) * \log{(1-A_3)}]$ (remember, $\hat{Y}$ and $A_3$ are equivalent)



In [ ]:
def compute_gradient_layer_3(y_hat, Y, activations):
    dc_dW3 = activations["A2"] @ ((1/Y.shape[1]) * (y_hat - Y)).T
    dc_dB3 = np.sum(((1/Y.shape[1]) * (y_hat - Y)), axis=1, keepdims=True)
    return dc_dW3, dc_dB3

def compute_gradient_layer_2(y_hat, Y, weights, activations):
    s1 = ((1/Y.shape[1]) * (y_hat - Y)) 
    s2 = weights["W3"]
    s3 = s1.T @ s2
    s4 = activations["A2"] @ s3
    s5 = (1 - activations["A2"])
    s6 = s4 @ s5
    dc_dW2 = s6 @ activations["A1"].T
    dc_dB2 = np.sum(s6, axis=1, keepdims=True)
    return dc_dW2, dc_dB2

def compute_gradient_layer_1(y_hat, Y, weights, activations):
    s1 = ((1/Y.shape[1]) * (y_hat - Y)) 
    s2 = weights["W3"]
    s3 = s1.T @ s2
    s4 = activations["A2"] @ s3
    s5 = (1 - activations["A2"])
    s6 = s4 @ s5
    s7 = s6.T @ weights["W2"]
    s8 = s7.T * activations["A1"]
    s9 = (1 - activations["A1"])
    s10 = s8 * s9
    dc_dW1 = s10 @ X.T
    dc_dB1 = np.sum(s10, axis=1, keepdims=True)
    return dc_dW1, dc_dB1

In [ ]:
A0, Y = prepare_data()
y_hat, activations, weights = feed_forward(A0)

In [ ]:
def prepare_data():
    X = np.array([
    [150, 70],
    [254, 73],
    [312, 68],
    [120, 60],
    [154, 61],
    [212, 65],
    [216, 67],
    [145, 67],
    [184, 64],
    [130, 69]
])

    A0 = X.T

    y = np.array([
    0,
    1, 
    1,
    0,
    0,
    1,
    1,
    0,
    1,
    0
])
    m = X.shape[0]
    
    # we need to reshape to a n^[3] x m matrix
    Y = y.reshape(n[3], m)
    return A0, Y

In [ ]:
learning_rate = 0.1
epochs = 1000

In [ ]:
costs = []

for epoch in range(epochs):
    print(f"... Running epoch {epoch} ...")

    # Forward pass
    y_hat, activations, weights = feed_forward(A0)
    loss = bce_loss(Y, y_hat)
    
    costs.append(loss)
    
    # Backward pass
    # Gradient compute
    dc_dW3, dc_dB3 = compute_gradient_layer_3(y_hat, Y, activations)
    dc_dW2, dc_dB2 = compute_gradient_layer_2(y_hat, Y, weights, activations)
    dc_dW1, dc_dB1 = compute_gradient_layer_1(y_hat, Y, weights, activations)

    # Backprop
    W3 = W3 - learning_rate * dc_dW3
    W2 = W2 - learning_rate * dc_dW2
    W1 = W1 - learning_rate * dc_dW1

    B3 = B3 - learning_rate * dc_dB3
    B2 = B2 - learning_rate * dc_dB2
    B1 = B1 - learning_rate * dc_dB1

    print(f"Cost for epoch {epoch} : {loss}")

In [ ]:
# TODO: explain why we didn't do softmax